### SQL DB

In [21]:
import sqlite3
import pandas as pd
import os
from typing import List,Dict,Any
from langchain_core.documents import Document

In [2]:
os.makedirs('data/sql', exist_ok=True)

In [3]:
conn=sqlite3.connect('data/sql/database.db')
cursor = conn.cursor()

In [4]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        NAME TEXT,
        ROLE TEXT,
        DEPARTMENT TEXT,
        SALARY REAL)
''')

In [5]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS PROJECT (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        NAME TEXT,
        STATUS TEXT,
        BUDGET REAL,
        LEAD_ID INTEGER
    )
''')

In [6]:
employees = [
    (1, 'John Doe', 'Senior Developer', 'Engineering', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000),
    (3, 'Mike Johnson', 'Product Manager', 'Product', 110000),
    (4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline', 'Completed', 80000, 2),
    (3, 'Customer Portal', 'Planning', 200000, 3),
    (4, 'ML Platform', 'Active', 250000, 2)
]

In [8]:
cursor.executemany('INSERT OR REPLACE INTO EMPLOYEES VALUES (?,?,?,?,?)',employees)
cursor.executemany('INSERT OR REPLACE INTO PROJECT VALUES (?,?,?,?,?)',projects)

In [9]:
cursor.execute('select * from employees')

In [10]:
conn.commit()
conn.close()

In [11]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

In [17]:
### Method-1 SQL DATABASE UTILITIES
db=SQLDatabase.from_uri('sqlite:///data/sql/database.db')
print(db.get_table_info())


CREATE TABLE "PROJECT" (
	"ID" INTEGER, 
	"NAME" TEXT, 
	"STATUS" TEXT, 
	"BUDGET" REAL, 
	"LEAD_ID" INTEGER, 
	PRIMARY KEY ("ID")
)

/*
3 rows from PROJECT table:
ID	NAME	STATUS	BUDGET	LEAD_ID
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline	Completed	80000.0	2
3	Customer Portal	Planning	200000.0	3
*/


CREATE TABLE employees (
	"ID" INTEGER, 
	"NAME" TEXT, 
	"ROLE" TEXT, 
	"DEPARTMENT" TEXT, 
	"SALARY" REAL, 
	PRIMARY KEY ("ID")
)

/*
3 rows from employees table:
ID	NAME	ROLE	DEPARTMENT	SALARY
1	John Doe	Senior Developer	Engineering	95000.0
2	Jane Smith	Data Scientist	Analytics	105000.0
3	Mike Johnson	Product Manager	Product	110000.0
*/


In [18]:
print(db.get_usable_table_names())

['PROJECT', 'employees']


In [32]:
print('\n Custom SQL Processing')

def sql_to_document(db_path: str) -> List[Document]:
    """Convert a SQL database to a list of Documents"""
    conn=sqlite3.connect(db_path)
    cursor=conn.cursor()

    documents=[]
    cursor.execute("SELECT name from sqlite_master WHERE type='table'")
    tables=cursor.fetchall()
    for table in tables:
        table_name=table[0]
        # get table name
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns=cursor.fetchall()
        columns_name = [col[1] for col in columns]

        cursor.execute(f"SELECT * FROM {table_name}")
        rows=cursor.fetchall()

        table_content = f"Table: {table_name}"
        table_content += f"Columns: {' '.join(columns_name)}"
        table_content += f"\n Total records: {len(rows)} rows"

        table_content += "Sample records: \n"
        for row in rows[:5]:
            record = dict(zip(columns_name,row))
            table_content += f"\n{record}"

        doc=Document(
            page_content=table_content,
            metadata = {
                'source':db_path,
                'table_name':table_name,
                'table_content':table_content,
                'data_type':'sql_table'
            }
        )
        documents.append(doc)

    cursor.execute("""
        SELECT e.name, e.role, p.name as project_name, p.status
        FROM employees e
        JOIN project p ON e.id = p.lead_id
    """)

    relationship = cursor.fetchall()
    rel_content = "employees-project relationship"
    for rel in relationship:
        rel_content+=f"{rel[0]} -> {rel[1]} leads {rel[2]} - Status: {rel[3]}"

    rel_doc = Document(
        page_content=rel_content,
        metadata={
            'source':db_path,
            'data_type':'sql_relationship',
            'query':'employees-project relationship'
        }
    )
    documents.append(rel_doc)
    conn.close()
    return documents



 Custom SQL Processing


In [33]:
sql_to_document('data/sql/database.db')

[Document(metadata={'source': 'data/sql/database.db', 'table_name': 'employees', 'table_content': "Table: employeesColumns: ID NAME ROLE DEPARTMENT SALARY\n Total records: 4 rowsSample records: \n\n{'ID': 1, 'NAME': 'John Doe', 'ROLE': 'Senior Developer', 'DEPARTMENT': 'Engineering', 'SALARY': 95000.0}\n{'ID': 2, 'NAME': 'Jane Smith', 'ROLE': 'Data Scientist', 'DEPARTMENT': 'Analytics', 'SALARY': 105000.0}\n{'ID': 3, 'NAME': 'Mike Johnson', 'ROLE': 'Product Manager', 'DEPARTMENT': 'Product', 'SALARY': 110000.0}\n{'ID': 4, 'NAME': 'Sarah Williams', 'ROLE': 'DevOps Engineer', 'DEPARTMENT': 'Engineering', 'SALARY': 98000.0}", 'data_type': 'sql_table'}, page_content="Table: employeesColumns: ID NAME ROLE DEPARTMENT SALARY\n Total records: 4 rowsSample records: \n\n{'ID': 1, 'NAME': 'John Doe', 'ROLE': 'Senior Developer', 'DEPARTMENT': 'Engineering', 'SALARY': 95000.0}\n{'ID': 2, 'NAME': 'Jane Smith', 'ROLE': 'Data Scientist', 'DEPARTMENT': 'Analytics', 'SALARY': 105000.0}\n{'ID': 3, 'NAME'